In [ ]:
import os
import pandas as pd
import numpy as np
from fancyimpute import KNN   
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sn
from random import randrange, uniform
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import pylab
from scipy import stats
from sklearn.cross_validation import train_test_split
import statsmodels.api as sm
import warnings
from random import *    
from numpy import *
from scipy.stats import chisquare
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [ ]:
#Extracting Data
original_data = pd.read_csv('Data_Project_1.csv', encoding = ' iso-8859-1')

In [ ]:
original_data = original_data.reset_index(drop=True)
original_data.index += 1

In [ ]:
working_data=original_data

In [ ]:
df=working_data.drop(['dteday','instant'],axis=1)
working_data=working_data.drop(['dteday'],1)

In [ ]:
df_cat=df.copy()
df_num=df.copy()

In [ ]:
#Converting variables to categoric and numeric
num_names=['temp','atemp','hum','windspeed','casual','registered','cnt']
cat_names=['yr','season','mnth','holiday','weekday','workingday','weathersit']

df_num_sliced = df_num.loc[:,num_names]
df_cat_sliced = df_num.loc[:,cat_names]

for var in cat_names:
    df_cat[var] = df_cat[var].astype("category")
    df_cat_sliced[var] = df_cat_sliced[var].astype("category")

In [ ]:
#Missing value analysis
missing_val = pd.DataFrame(df.isnull().sum())
missing_val

In [ ]:
#Visualizing the Raw Data
fig,(ax1,ax2,ax3,ax4, ax5, ax6)= plt.subplots(nrows=6)
fig.set_size_inches(13,25)
cnt_by_mnth = pd.DataFrame(df_cat.groupby("mnth")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_mnth,x="mnth",y="cnt",ax=ax1)

cnt_by_season = pd.DataFrame(df_cat.groupby(["season"])["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_season,x="season",y="cnt",ax=ax2)

cnt_by_weekday = pd.DataFrame(df_cat.groupby("weekday")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_weekday,x="weekday",y="cnt",ax=ax3)

cnt_by_weathersit = pd.DataFrame(df_cat.groupby("weathersit")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_weathersit,x="weathersit",y="cnt",ax=ax4)

cnt_by_yr = pd.DataFrame(df_cat.groupby("yr")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_yr,x="yr",y="cnt",ax=ax5)

transformed = pd.melt(df_cat[["yr","casual","registered"]], id_vars=['yr'], value_vars=['casual', 'registered'])
cnt_by_user= pd.DataFrame(transformed.groupby(["yr","variable"],sort=True)["value"].mean()).reset_index()
sn.pointplot(x=cnt_by_user["yr"], y=cnt_by_user["value"],hue=cnt_by_user["variable"],hue_order=["casual","registered"], data=transformed, join=True,ax=ax6)
ax6.set(xlabel='Year', ylabel='Users Count',label='big')

In [ ]:
#outlier analysis
#visualizing outliers (boxplot)
fig, axes = plt.subplots(nrows=3,ncols=2)
fig.set_size_inches(12,10)
sn.boxplot(y='hum', x='mnth',data=df_cat, width=.5,palette="colorblind",ax=axes[0][0])
sn.boxplot(y='windspeed', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[0][1])
sn.boxplot(y='atemp', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[1][0])
sn.boxplot(y='cnt', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[1][1])
sn.boxplot(y='casual', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[2][0])
sn.boxplot(y='registered', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[2][1])

In [ ]:
#Visualizing distribution 
ax8 = original_data.plot(kind='scatter',x='instant', y='registered')
ax9 = original_data.plot(kind='scatter',x='instant', y='casual')
ax10 = original_data.plot(kind='scatter',x='instant', y='cnt')

In [ ]:
#Histograms before removing outliers
a4_dims = (14, 14)
fig, axes = plt.subplots(nrows=3,ncols=2,figsize=a4_dims)
sn.set(color_codes=True)
sn.set(style="white", palette="muted")
sn.distplot(df_num['hum'], ax=axes[0][0])
sn.distplot(df_num['atemp'], ax=axes[0][1])
sn.distplot(df_num['registered'], ax=axes[1][0])
sn.distplot(df_num['windspeed'], ax=axes[1][1])
sn.distplot(df_num['cnt'], ax=axes[2][0])
sn.distplot(df_num['casual'], ax=axes[2][1])



# Outlier analysis

In [ ]:
#Detecting outlier and replacing it with NA

q75_w, q25_w = np.percentile(df['windspeed'], [75 ,25])


iqr_w = q75_w - q25_w


minimum_w = q25_w - (iqr_w*1.5)
maximum_w = q75_w + (iqr_w*1.5)

q75_h, q25_h = np.percentile(df['hum'], [75 ,25])


iqr_h = q75_h - q25_h


minimum_h = q25_h - (iqr_h*1.5)
maximum_h = q75_h + (iqr_h*1.5)

q75_cnt, q25_cnt = np.percentile(df['casual'], [75 ,25])


iqr_cnt = q75_cnt - q25_cnt


minimum_cnt = q25_cnt - (iqr_cnt*1.5)
maximum_cnt = q75_cnt + (iqr_cnt*1.5)

q75_atemp, q25_atemp = np.percentile(df_num['atemp'], [75 ,25])


iqr_atemp = q75_atemp - q25_atemp


minimum_atemp = q25_atemp - (iqr_atemp*1.5)
maximum_atemp = q75_atemp + (iqr_atemp*1.5)


df_cat.loc[df_cat['hum'] < minimum_h, 'hum'] = np.nan
df_cat.loc[df_cat['hum'] > maximum_h, 'hum'] = np.nan
df_num.loc[df_num['hum'] < minimum_h, 'hum'] = np.nan
df_num.loc[df_num['hum'] > maximum_h, 'hum'] = np.nan
df_cat.loc[df_cat['windspeed'] < minimum_w, 'windspeed'] = np.nan
df_cat.loc[df_cat['windspeed'] > maximum_w, 'windspeed'] = np.nan
df_num.loc[df_num['windspeed'] < minimum_w, 'windspeed'] = np.nan
df_num.loc[df_num['windspeed'] > maximum_w, 'windspeed'] = np.nan
df_cat.loc[df_cat['casual'] < minimum_cnt, 'casual'] = np.nan
df_cat.loc[df_cat['casual'] > maximum_cnt, 'casual'] = np.nan
df_num.loc[df_num['casual'] < minimum_cnt, 'casual'] = np.nan
df_num.loc[df_num['casual'] > maximum_cnt, 'casual'] = np.nan
df_cat.loc[df_cat['atemp'] < minimum_atemp, 'atemp'] = np.nan
df_cat.loc[df_cat['atemp'] > maximum_atemp, 'atemp'] = np.nan
df_num.loc[df_num['atemp'] < minimum_atemp, 'atemp'] = np.nan
df_num.loc[df_num['atemp'] > maximum_atemp, 'atemp'] = np.nan

In [ ]:
maximum_cnt

In [ ]:
# #Calculating missing value after outlier analysis
missing_val = pd.DataFrame(df_cat.isnull().sum())
missing_val

In [ ]:
df_cat=df_cat.dropna()
df_num=df_num.dropna()

In [ ]:
#Comparing before and after outlier analysis scatter plot
df_cat_sc= df_cat
df_cat_sc['instant']= original_data['instant']
ax9_new = df_cat_sc.plot(kind='scatter',x='instant', y='casual', color="red")
ax10_new = df_cat_sc.plot(kind='scatter',x='instant', y='cnt')
ax9 = original_data.plot(kind='scatter',x='instant', y='casual',color="red")
ax10 = original_data.plot(kind='scatter',x='instant', y='cnt')

In [ ]:
df_cat=df_cat.drop(['instant'],1) 

In [ ]:
#Visualizing the data that has been removed by deleting outliers of variable casual
df.loc[df['casual'] > 2266]

#### Removal of outliers had only cost major drop to the variable 'weekday'

##### Seeing the effect on weekday before and after outlier play

In [ ]:
df_cat.groupby(["yr","weekday"])["cnt"].mean()

In [ ]:
df.groupby(["yr","weekday"])["cnt"].mean()

In [ ]:
df_cat['weekday'].value_counts()

In [ ]:
df['weekday'].value_counts()

In [ ]:
#Histograms after removing outliers
a4_dims = (14, 14)
fig, axes = plt.subplots(nrows=3,ncols=2,figsize=a4_dims)
sn.set(color_codes=True)
sn.set(style="white", palette="muted")
sn.distplot(df_num['hum'], ax=axes[0][0])
sn.distplot(df_num['atemp'], ax=axes[0][1])
sn.distplot(df_num['registered'], ax=axes[1][0])
sn.distplot(df_num['windspeed'], ax=axes[1][1])
sn.distplot(df_num['cnt'], ax=axes[2][0])
sn.distplot(df_num['casual'], ax=axes[2][1])

In [ ]:
original_data.groupby(["season"])["mnth"].unique().to_frame()

In [ ]:
#outlier analysis
#visualizing outliers through boxplot after droppping
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(12, 10)
sn.boxplot(y='hum', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[0][0])
sn.boxplot(y='windspeed', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[0][1])
sn.boxplot(y='atemp', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[1][0])
sn.boxplot(y='casual', x='mnth',data=df_cat, width=0.5,palette="colorblind",ax=axes[1][1])

In [ ]:
df_num_sliced = df_num.iloc[:,7:15]
df_cat_sliced = df_cat.iloc[:,0:7]

In [ ]:
df_cat_sliced.dtypes

In [ ]:
#for var in cat_names:
 #   df_cat[var] = df_cat[var].astype("category")
  #  df_cat_sliced[var] = df_cat_sliced[var].astype("category")

In [ ]:
df_num = df_num.reset_index(drop=True)
df_num.index += 1
df_cat = df_cat.reset_index(drop=True)
df_cat.index += 1
df_num_sliced = df_num_sliced.reset_index(drop=True)
df_num_sliced.index += 1
df_cat_sliced = df_cat_sliced.reset_index(drop=True)
df_cat_sliced.index += 1

In [ ]:
# Converting the target continious variable to categorical variable by binning for various tests
bins = [0,1000, 2000,3000, 4000,5000, 6000,7000, 8000,9000]
labels = [1,2,3,4,5,6,7,8,9]
df_cat['cnt_binned'] = pd.cut(df_num['cnt'], bins=bins, labels=labels)
df_num['cnt_binned'] = pd.cut(df_num['cnt'], bins=bins, labels=labels)
df_cat_sliced['cnt_binned'] = pd.cut(df_num['cnt'], bins=bins, labels=labels)

# Feature Selection

In [ ]:
#Feature Selection using Correlation, ANOVA, and Chi-Square test

## 1) Correlation

correlations = df_num_sliced.corr()
correlations
correlations.style.background_gradient()

In [ ]:
## 2) Chi-square test of each variable with other (When dependent variable has been converted to categorical through binning)


factors_paired_bin = [(i,j) for i in df_cat_sliced.columns.values for j in df_cat_sliced.columns.values] 

chi2_bin, p_values_bin =[], []

for f in factors_paired_bin:
    if f[0] != f[1]:
             chitest_bin = chi2_contingency(pd.crosstab(df_cat_sliced[f[0]], df_cat_sliced[f[1]]))   
             chi2_bin.append(chitest_bin[0])
             p_values_bin.append(chitest_bin[1])
            
    else:      
             chi2_bin.append(0)
             p_values_bin.append(0)

chi2_bin = np.array(chi2_bin).reshape((8,8))
chi2_df_bin = pd.DataFrame(chi2_bin, index=df_cat_sliced.columns.values, columns=df_cat_sliced.columns.values)
p_values_bin = np.array(p_values_bin).reshape((8,8)) # shaping it as a matrix
p_values_bin = pd.DataFrame(p_values_bin, index=df_cat_sliced.columns.values, columns=df_cat_sliced.columns.values)
p_values_bin

In [ ]:
#ANOVA Analysis
import statsmodels.api as sm
from statsmodels.formula.api import ols


cw_lm=ols('cnt ~ C(yr)+C(holiday)+C(workingday)+ C(mnth)+C(weekday)+ C(weathersit)+C(season)', data=df_cat).fit() 
print(sm.stats.anova_lm(cw_lm, typ=2))

In [ ]:
#Checking the amount of data for category holiday
df_cat['holiday'].value_counts()

In [ ]:
#checking the importance of variable holiday
df_cat.groupby(["yr","holiday"])["cnt"].mean()

In [ ]:
# Visualization of data through bar graphs after removing outliers
fig,(ax1,ax2,ax3,ax4, ax5, ax6)= plt.subplots(nrows=6)
fig.set_size_inches(11,20)
cnt_by_mnth = pd.DataFrame(df_cat.groupby("mnth")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_mnth,x="mnth",y="cnt",ax=ax1)

cnt_by_season = pd.DataFrame(df_cat.groupby("season")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_season,x="season",y="cnt",ax=ax2)

cnt_by_weekday = pd.DataFrame(df_cat.groupby("weekday")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_weekday,x="weekday",y="cnt",ax=ax3)

cnt_by_weathersit = pd.DataFrame(df_cat.groupby("weathersit")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_weathersit,x="weathersit",y="cnt",ax=ax4)

cnt_by_yr = pd.DataFrame(df_cat.groupby("yr")["cnt"].mean()).reset_index()
sn.barplot(data=cnt_by_yr,x="yr",y="cnt",ax=ax5)

transformed = pd.melt(df_cat[["yr","casual","registered"]], id_vars=['yr'], value_vars=['casual', 'registered'])
cnt_by_user= pd.DataFrame(transformed.groupby(["yr","variable"],sort=True)["value"].mean()).reset_index()
sn.pointplot(x=cnt_by_user["yr"], y=cnt_by_user["value"],hue=cnt_by_user["variable"],hue_order=["casual","registered"], data=transformed, join=True,ax=ax6)
ax6.set(xlabel='Year', ylabel='Users Count',label='big')

In [ ]:
#mnth_names = ["January","February","March","April","May","June","July","August","September","October","November","December"]
#weekday_names = ["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"]
#season_names = ["Spring","Summer","fall", "winter"]
#weather_types = ["lev_a","lev_b","lev_c"]

In [ ]:
#Dropping unnecessary variables according to the model

drop_features_for_lm=["casual","registered","temp"]
drop_features_for_dt_classification_after_chi=["casual","registered","temp"]
drop_features_for_rf_classification_after_chi=["casual","registered","temp"]

# Model development

In [ ]:
#Splitting data into train and test
random.seed(20)
train_num, test_num = train_test_split(df_cat, test_size=0.2)
train_num.dtypes

In [ ]:
#df5tr['yr'] = df5tr['yr'].astype('category')
#df5tr['holiday'] = df5tr['holiday'].astype('category')
#df5tr['workingday'] = df5tr['workingday'].astype('category')
#df5te['yr'] = df5te['yr'].astype('category')
#df5te['holiday'] = df5te['holiday'].astype('category')
#df5te['workingday'] = df5te['workingday'].astype('category')


In [ ]:
train_num = train_num.drop(['casual','registered','temp'], axis=1)
test_num = test_num.drop(['casual','registered','temp'], axis=1)

## Decision Tree

In [ ]:
# 1) Decision tree regressor
                 
                 #a)Creating no bins for  categorical variables, and keeping numerical and other variables as it is
        
fit_dt = DecisionTreeRegressor(max_depth=10).fit(train_num.iloc[:,0:10], train_num.iloc[:,10])

In [ ]:
predictions_dt = fit_dt.predict(test_num.iloc[:,0:10])

In [ ]:
def MAPE(y_true, y_pred): 
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
    
    return mape

MAPE(test_num.iloc[:,10], predictions_dt)

### Checking the reason for high MAPE

In [ ]:
dataset = pd.DataFrame({'pred':predictions_dt})

In [ ]:
col_1 = dataset
col_1

In [ ]:
col_2=test_num.iloc[:,10].to_frame()
col2=col_2.reset_index().drop('index',1)

In [ ]:
col_2=col_2.reset_index()

In [ ]:
col_2=col_2.drop('index',1)

In [ ]:
col_2

In [ ]:
new_col= pd.DataFrame(columns=['subt'])


In [ ]:
new_col['subt']= (col_2['cnt']-col_1['pred'])/(col_2['cnt'])

In [ ]:
#Values of absolute percentage error
new_col = new_col.sort_values('subt', ascending = True)
new_col

In [ ]:
minimum_cnt

In [ ]:

df_num.loc[df_num['casual'] > 2200]

In [ ]:
df_num= df_num.sort_values('cnt', ascending = True)
df_num

In [ ]:
#Looking for the unnatural data 
df_cat.loc[df_cat['casual'] == 2]

In [ ]:
#Removing the row from dataset
df_num=df_num[df_num.casual != 2]
df_cat=df_cat[df_cat.casual != 2]


train_num=train_num[train_num.cnt!=22]

test_num=test_num[test_num.cnt!=22]

# Remodelling

## Decision Tree

In [ ]:
# 1) Decision tree regressor
                 
                 #a)Creating no bins for  categorical variables, and keeping numerical and other variables as it is
        
fit_dt = DecisionTreeRegressor(max_depth=10).fit(train_num.iloc[:,0:10], train_num.iloc[:,10])

In [ ]:
predictions_dt = fit_dt.predict(test_num.iloc[:,0:10])

In [ ]:
def MAPE(y_true, y_pred): 
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
    
    return mape

MAPE(test_num.iloc[:,10], predictions_dt)

In [ ]:
#Creating dot file to visualise tree  #http://webgraphviz.com/

dotfile = open("pro.dot", 'w')
dt_export = tree.export_graphviz(fit_dt, out_file=dotfile, feature_names = train_num_vis.columns)

## Linear Regression

In [ ]:
#REGRESSION MODELS

# 2) Linear regression: 

    #a)Creating dummies for each categorical variable, and keeping numerical variable as it is
    
weathersit_dummy = pd.get_dummies(df_num.weathersit) 
season_dummy = pd.get_dummies(df_num.season) 
mnth_dummy = pd.get_dummies(df_num.mnth) 
weekday_dummy = pd.get_dummies(df_num.weekday)


In [ ]:
mnth_dummy.columns=['mnth_1','mnth_2','mnth_3','mnth_4','mnth_5','mnth_6','mnth_7','mnth_8','mnth_9','mnth_10','mnth_11','mnth_12']
weekday_dummy.columns=['weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6','weekday_7']
season_dummy.columns=['season_1','season_2','season_3','season_4']
weathersit_dummy.columns=['weathersit_1','weathersit_1','weathersit_3']


df2tr = pd.merge(train_num, season_dummy ,left_index=True, right_index=True)
df3tr = pd.merge(df2tr, weathersit_dummy ,left_index=True, right_index=True)
df4tr = pd.merge(df3tr, weekday_dummy ,left_index=True, right_index=True)
df5tr = pd.merge(df4tr, mnth_dummy ,left_index=True, right_index=True)

df2te = pd.merge(test_num, season_dummy ,left_index=True, right_index=True)
df3te = pd.merge(df2te, weathersit_dummy ,left_index=True, right_index=True)
df4te = pd.merge(df3te, weekday_dummy ,left_index=True, right_index=True)
df5te = pd.merge(df4te, mnth_dummy ,left_index=True, right_index=True)




In [ ]:
df5tr = df5tr.drop(['season','mnth','weekday','weathersit','casual','registered','temp'], axis=1)
df5te = df5te.drop(['season','mnth','weekday','weathersit','casual','registered','temp'], axis=1)
df5tr=df5tr[df5tr.cnt != 22]
df5te=df5te[df5te.cnt != 22]

In [ ]:
df5tr['count']=df5tr['cnt']
df5tr['cnt_bin_temp']=df5tr['cnt_binned']
df5tr=df5tr.drop(['cnt','cnt_binned'],1)
df5te['count']=df5te['cnt']
df5te['cnt_bin_temp']=df5te['cnt_binned']
df5te=df5te.drop(['cnt','cnt_binned'],1)

In [ ]:
df5tr = df5tr.rename(columns={'count': 'cnt', 'cnt_bin_temp': 'cnt_binned'})
df5te = df5te.rename(columns={'count': 'cnt', 'cnt_bin_temp': 'cnt_binned'})

In [ ]:
df5tr=df5tr.astype('float')
df5te=df5te.astype('float')


In [ ]:
model_lr_dummies = sm.OLS(df5tr.iloc[:,32], df5tr.iloc[:,0:32].astype('float')).fit()

In [ ]:
model_lr_dummies.summary()

In [ ]:
predictions_lr_dummies = model_lr_dummies.predict(df5te.iloc[:,0:32])

In [ ]:
def MAPE(y_true, y_pred): 
    mape_lr_dummies = np.mean(np.abs((y_true - y_pred) / y_true))*100
    return mape_lr_dummies
#Calculate MAPE
MAPE(df5te.iloc[:,32], predictions_lr_dummies)

In [ ]:
# 2) Linear regression: 

    #b)Creating dummies for only ordinal variable, and keeping numerical and other variables as it is

In [ ]:
weathersit_dummy = pd.get_dummies(df_num.weathersit)
weathersit_dummy.columns=['weathersit_1','weathersit_2','weathersit_3']
df5tror = pd.merge(train_num, weathersit_dummy ,left_index=True, right_index=True)
df5tror=df5tror.drop(['cnt','cnt_binned','weathersit'], 1)
df5tror['cnt']=df5tr['cnt']
df5tror['cnt_binned']=df5tr['cnt_binned']

In [ ]:
df5teor = pd.merge(test_num, weathersit_dummy ,left_index=True, right_index=True)
df5teor=df5teor.drop(['cnt','cnt_binned','weathersit'], 1)
df5teor['cnt']=df5te['cnt']
df5teor['cnt_binned']=df5te['cnt_binned']

In [ ]:
#df5tr['yr'] = df5tr['yr'].astype('category')
#df5tr['holiday'] = df5tr['holiday'].astype('category')
#df5tr['workingday'] = df5tr['workingday'].astype('category')
#df5te['yr'] = df5te['yr'].astype('category')
#df5te['holiday'] = df5te['holiday'].astype('category')
#df5te['workingday'] = df5te['workingday'].astype('category')

In [ ]:
model_lr_or_dummies = sm.OLS(df5tror.iloc[:,12], df5tror.iloc[:,0:12].astype(float)).fit()

In [ ]:
model_lr_or_dummies.summary()

In [ ]:
predictions_lr_or_dummies = model_lr_or_dummies.predict(df5teor.iloc[:,0:12])
#df_num.sort_values('cnt',ascending=True)

In [ ]:
MAPE(df5teor.iloc[:,12], predictions_lr_or_dummies)

In [ ]:
# 2) Linear regression: 

    #c)Creating no bins for  categorical variables, and keeping numerical and other variables as it is

In [ ]:
model = sm.OLS(train_num.iloc[:,10], train_num.iloc[:,0:10].astype(float)).fit()

In [ ]:
model.summary()

In [ ]:
predictions = model.predict(test_num.iloc[:,0:10])

In [ ]:
def MAPE(y_true, y_pred): 
    mape = np.mean(np.abs((y_true - y_pred) / y_true))*100
    return mape
#Calculate MAPE
MAPE(test_num.iloc[:,10], predictions)

## Random Forest

In [ ]:
    # 3) Random forest regressor
                 
                 #a)Creating bins for only ordinal variable, and keeping numerical and other variables as it is
rf_model_or = RandomForestRegressor(n_estimators = 20).fit(df5tror.iloc[:,0:12],df5tror.iloc[:,12])

In [ ]:
rf_predictions_or = rf_model_or.predict(df5teor.iloc[:,0:12])

In [ ]:
MAPE(df5teor.iloc[:,12] , rf_predictions_or)

In [ ]:
# 3) Random forest regressor
                 
                 #b)Creating no bins for  categorical variables, and keeping numerical and other variables as it is
rf_model = RandomForestRegressor(n_estimators = 80).fit(train_num.iloc[:,0:10], train_num.iloc[:,10])

In [ ]:
rf_predictions = rf_model.predict(test_num.iloc[:,0:10])

In [ ]:
MAPE(test_num.iloc[:,10],rf_predictions)

In [ ]:
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

In [ ]:
train_num_vis=train_num.drop(['cnt','cnt_binned'], 1)

In [ ]:
#Plot of the threshold trees required for random forest
estimators = np.arange(10, 400, 10)
scores = []
for n in estimators:
    rf_model.set_params(n_estimators=n)
    rf_model.fit(train_num.iloc[:,0:10],train_num.iloc[:,10])
    scores.append(rf_model.score(test_num.iloc[:,0:10],test_num.iloc[:,10]))
plt.title("Effect of n_estimators")
plt.xlabel("n_estimator")
plt.ylabel("score")
plt.plot(estimators, scores)

In [ ]:
# 4) KNN 

KNN_model = KNeighborsRegressor(n_neighbors = 5).fit(train_num.iloc[:,0:10], train_num.iloc[:,10])

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
KNN_Predictions = KNN_model.predict(test_num.iloc[:,0:10])

In [ ]:
MAPE(test_num.iloc[:,10], KNN_Predictions)

# Re-Modelling by imputing outliers through random forest

###  Imputation of outliers by categories' means

In [ ]:
#Detecting outliers and replacing with NAs

q75_w, q25_w = np.percentile(df['windspeed'], [75 ,25])


iqr_w = q75_w - q25_w


minimum_w = q25_w - (iqr_w*1.5)
maximum_w = q75_w + (iqr_w*1.5)

q75_h, q25_h = np.percentile(df['hum'], [75 ,25])


iqr_h = q75_h - q25_h


minimum_h = q25_h - (iqr_h*1.5)
maximum_h = q75_h + (iqr_h*1.5)

q75_cnt, q25_cnt = np.percentile(df['casual'], [75 ,25])


iqr_cnt = q75_cnt - q25_cnt


minimum_cnt = q25_cnt - (iqr_cnt*1.5)
maximum_cnt = q75_cnt + (iqr_cnt*1.5)



df.loc[df['hum'] < minimum_h, 'hum'] = np.nan
df.loc[df['hum'] > maximum_h, 'hum'] = np.nan
df.loc[df['windspeed'] < minimum_w, 'windspeed'] = np.nan
df.loc[df['windspeed'] > maximum_w, 'windspeed'] = np.nan
df.loc[df['casual'] < minimum_cnt, 'casual'] = np.nan
df.loc[df['casual'] > maximum_cnt, 'casual'] = np.nan

In [ ]:
#Imputing means of weekday category to cnt
df.loc[(df['weekday'] == 6) & pd.isnull(df['casual']) , 'cnt'] = 5732 
df.loc[(df['weekday'] == 0) & pd.isnull(df['casual']) , 'cnt'] = 5036  

In [ ]:
df=df.drop('casual', axis=1)

In [ ]:
#Dropping rest of the NAs from hum and windspeed as they are randomly distributed
df=df.dropna()

In [ ]:
df=df.drop(['atemp','registered'], 1)

In [ ]:
random.seed(20)

train_num_imp, test_num_imp = train_test_split(df, test_size=0.2)

In [ ]:
rf_model_imp = RandomForestRegressor(n_estimators = 51).fit(train_num_imp.iloc[:,0:10], train_num_imp.iloc[:,10])

In [ ]:
rf_predictions_imp = rf_model_imp.predict(test_num_imp.iloc[:,0:10])

In [ ]:
MAPE(test_num_imp.iloc[:,10],rf_predictions_imp)

In [ ]:
##Plot of the threshold trees required for re-modelled random forest
estimators = np.arange(10, 400, 10)
scores = []
for n in estimators:
    rf_model.set_params(n_estimators=n)
    rf_model.fit(train_num_imp.iloc[:,0:10],train_num_imp.iloc[:,10])
    scores.append(rf_model.score(test_num_imp.iloc[:,0:10],test_num_imp.iloc[:,10]))
plt.title("Effect of n_estimators")
plt.xlabel("n_estimator")
plt.ylabel("score")
plt.plot(estimators, scores)